In [1]:
project = "gitops-project"

### Deploy Stream Processing

In [2]:
import v3io.dataplane

In [3]:
v3io_client = v3io.dataplane.Client()

In [4]:
stream_path=f'{project}/model-endpoints/stream'
container='projects'

stream_path

'gitops-project/model-endpoints/stream'

In [5]:
v3io_client.stream.create(container='projects', stream_path=f'{project}/model-endpoints/stream', shard_count=1, raise_for_status=[204, 409])

In [6]:
v3io_client.stream.create(container='projects', stream_path=f'{project}/model-endpoints/log_stream', shard_count=1, raise_for_status=[204, 409])

In [7]:
import os

from mlrun import import_function
from mlrun.platforms import mount_v3io
from mlrun.runtimes import RemoteRuntime
import json

fn: RemoteRuntime = import_function("hub://model_monitoring_stream")

# Configures the nuclio trigger. See nuclio triggers https://nuclio.io/docs/latest/reference/triggers/.
fn.add_v3io_stream_trigger(
    stream_path=f"projects/{project}/model-endpoints/stream",
    name="monitoring_stream_trigger",
)

fn.set_env("MODEL_MONITORING_PARAMETERS", json.dumps(
    {"project": project, "v3io_framesd": os.environ.get("V3IO_FRAMESD")}))

fn.metadata.project = project
fn.apply(mount_v3io())

# Deploy the function
fn.deploy()

> 2023-06-28 18:05:44,830 [info] Starting remote function deploy
2023-06-28 18:05:45  (info) Deploying function
2023-06-28 18:05:45  (info) Building
2023-06-28 18:05:46  (info) Staging files and preparing base images
2023-06-28 18:05:46  (info) Building processor image
2023-06-28 18:06:21  (info) Build complete
2023-06-28 18:06:42  (info) Function deploy complete
> 2023-06-28 18:06:47,021 [info] successfully deployed function: {'internal_invocation_urls': ['nuclio-gitops-project-model-monitoring-stream.default-tenant.svc.cluster.local:8080'], 'external_invocation_urls': ['gitops-project-model-monitoring-stream-gitops-project.default-tenant.app.cst-353.iguazio-cd1.com/']}


'http://gitops-project-model-monitoring-stream-gitops-project.default-tenant.app.cst-353.iguazio-cd1.com/'

### Deploy Batch Processing

In [8]:
from mlrun import code_to_function, mount_v3io

In [14]:
fn = code_to_function(
    name="model-monitoring-batch",
    kind="job",
    image="mlrun/mlrun",
    filename="model_monitoring_batch.py",
    handler="handler",
)
fn.metadata.project = project
fn.apply(mount_v3io())

# Run the function as an MLRun job
fn.run(name='model-monitoring-batch', schedule="*/10 * * * *",
       params={"project": project})

> 2023-06-28 18:13:20,642 [info] Storing function: {'name': 'model-monitoring-batch', 'uid': 'bf314462e83a40c1952aecbc88bf3ce1', 'db': 'http://mlrun-api:8080'}
> 2023-06-28 18:13:20,982 [info] task schedule modified: {'schedule': '*/10 * * * *', 'project': 'gitops-project', 'name': 'model-monitoring-batch'}
